<a href="https://colab.research.google.com/github/aadilganigaie/ChatAnalyzer/blob/main/langchainchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -qqq gradio==3.48.0 --progress-bar off

  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [ ]:
!pip -q install pypdf
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
#!pip install -qqq gradio==3.48.0 --progress-bar off
!pip install -qqq transformers==4.33.2 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq chromadb==0.4.10 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.14.0 --progress-bar off
!pip install -qqq optimum==1.13.1 --progress-bar off
!pip install -qqq auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!pip install -qqq unstructured==0.10.16 --progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into acc

In [ ]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Load and process the text files
#loader = DirectoryLoader("google drive path or local folder path", glob = "./*.pdf, loader_cls = pypdf")
# loader = TextLoader('single_text_file.txt')
#loader = DirectoryLoader('/content/drive/MyDrive/UoM', glob="./*.pdf", loader_cls= Py)
loader = PyPDFDirectoryLoader('/content/drive/MyDrive/Rawmaterial', glob="./*.pdf")
docs = loader.load()

In [ ]:
len(docs)

208

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

#query_result = embeddings.embed_query(texts[0].page_content)
#print(len(query_result))

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [ ]:
#splitting the text into samller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

"TheBloke/Llama-2-13b-Chat-GPTQ"

In [ ]:
MODEL_NAME = "TheBloke/Llama-2-13b-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings, persist_directory="db")
#results = db.similarity_search("how to contact institute of informatics?", k=2)
#print(results[0].page_content)

Use the following information to answer the question at the end.

template = """
<s>[INST] <<SYS>>
Act as an smart chatbot generate the questions .
<</SYS>>

{context}

{question} [/INST]
"""

In [ ]:
#Question Gneration
from langchain.chains import RetrievalQA

template = """
<s>[INST] <<SYS>>
Act Smart Teacher Please generate questions for the specified topic, subject, and difficulty level. If you have any specific requirements or formats for the questions, kindly mention them.

<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

'''result = qa_chain(
    "give me the contact details of rector of the university and also tell when was the university established?"
)
print(result["result"].strip())'''

'result = qa_chain(\n    "give me the contact details of rector of the university and also tell when was the university established?"\n)\nprint(result["result"].strip())'

In [ ]:
#Evaluate
from langchain.chains import RetrievalQA

template = """
<s>[INST] <<SYS>>
As an expert in sustainability, Circular Economy, Recycling, Raw Material Research, and simulation and modeling of raw materials, please provide insightful answers to the following questions. Ensure that your responses are detailed and reflect the latest advancements and best practices in the field. If necessary, include relevant data, research findings, and case studies to support your answers.

<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

result = qa_chain(
    "how recycling is affecting the circular economy?"
)
print(result["result"].strip())

As an expert in sustainability, Circular Economy, Recycling, Raw Material Research, and simulation and modeling of raw materials, I'd be happy to provide some insightful answers to your questions. Here are my responses:

1. How does recycling contribute to the circular economy?

Recycling plays a crucial role in the transition to a circular economy by reducing the need for primary raw materials, conserving resources, and decreasing waste generation. By reusing and refurbishing materials, recycling helps to close the loop between production and consumption, promoting the continuous use of resources rather than their disposal. This not only reduces the environmental impact of resource extraction but also saves energy and minimizes greenhouse gas emissions. Moreover, recycling creates jobs and stimulates economic growth while helping to secure the supply of critical raw materials.

2. What are the challenges facing the circular economy, particularly regarding critical raw materials?

The 

In [ ]:
from textwrap import fill
from IPython.display import display, HTML

# Function to get user input and display the model's response
def get_response():
    user_question = input("User: ")
    result = qa_chain(user_question)
    response_html = f"<div style='border: 1px solid black; padding: 10px; margin: 10px; background-color: #f0f0f0;'>{fill(result['result'].strip(), width=80)}</div>"
    display(HTML(response_html))

# Main loop for interacting with the chatbot
while True:
    get_response()

User:  provide suggestions on starting an enterprise for raw material management:


OutOfMemoryError: ignored

In [ ]:
from textwrap import fill

result = qa_chain(input("SustainAI ChatBot ---")
)
print(fill(result["result"].strip(), width=80))

SustainAI ChatBot ---provide suggestions on starting an enterprise for raw material management


OutOfMemoryError: ignored

In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.4/237.4 kB 4.5 MB/s eta 0:00:00


In [ ]:
#Question generation
import subprocess
from docx import Document

# Function to create a Word document
def create_and_print_document(output_text):
    # Create a new Word document
    doc = Document()
    doc.add_paragraph(output_text)
    doc.save("output.docx")


# Gradio interface function
def chatbot_interface(input_text):
    result = qa_chain(input_text)
    response = result["result"].strip()
    create_and_print_document(response)  # Create and print the document
    return response

# Create a Gradio Interface
gr.Interface(fn=chatbot_interface, inputs="text", outputs="text").launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://42c8ed4beabeae03de.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://42c8ed4beabeae03de.gradio.live


In [ ]:
#sustainility
from langchain.chains import RetrievalQA

template = """
<s>[INST] <<SYS>>
As an expert in sustainability, Circular Economy, Recycling, Raw Material Research, and simulation and modeling of raw materials, please provide insightful answers to the following questions. Ensure that your responses are detailed and reflect the latest advancements and best practices in the field. If necessary, include relevant data, research findings, and case studies to support your answers.

<</SYS>>

{context}

{question}

{response}
<s>After each response, please suggest practical tips for sustainability and raw material management, accompanied by a green symbol.</s>
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question", "response"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)


In [ ]:
# Text-based interface
while True:
    user_question = input("Ask a question: ")
    result = qa_chain(user_question)
    response = result["result"].strip()
    print("Model Response:", response)
    print("=" * 50)

Ask a question: how recycling effect the circular economy?


ValueError: ignored

In [ ]:
#Evaluation
#gradio
# Define a Gradio Interface
import gradio as gr
def chatbot_interface(input_text):
    result = qa_chain(input_text)
    response = result["result"].strip()
    return response

# Create a Gradio Interface
gr.Interface(fn=chatbot_interface, inputs="text", outputs="text").launch(debug = True, share = True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://558bee2e776afb31eb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
